#### Time Series Forecasting

- A time series is data collected periodically, over time.
- Time series forecasting is the task of predicting future data points, given some historical data. It is commonly used in a variety of tasks from weather forecasting, retail and sales forecasting, stock market prediction, and in behavior prediction (such as predicting the flow of car traffic over a day). There is a lot of time series data out there, and recognizing patterns in that data is an active area of machine learning research!

#### Machine Learning Workflow

This notebook approaches time series forecasting in a number of steps:

- Loading and exploring the data
- Creating training and test sets of time series
- Formatting data as JSON files and uploading to S3
- Instantiating and training a DeepAR estimator
- Deploying a model and creating a predictor
- Evaluating the predictor


#### Import packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

#### Load the data

In [2]:
n_lines = 10

with open('household_power_consumption.txt') as file:
    head = [next(file) for line in range(n_lines)]
    
display(head)

['Date;Time;Global_active_power;Global_reactive_power;Voltage;Global_intensity;Sub_metering_1;Sub_metering_2;Sub_metering_3\n',
 '16/12/2006;17:24:00;4.216;0.418;234.840;18.400;0.000;1.000;17.000\n',
 '16/12/2006;17:25:00;5.360;0.436;233.630;23.000;0.000;1.000;16.000\n',
 '16/12/2006;17:26:00;5.374;0.498;233.290;23.000;0.000;2.000;17.000\n',
 '16/12/2006;17:27:00;5.388;0.502;233.740;23.000;0.000;1.000;17.000\n',
 '16/12/2006;17:28:00;3.666;0.528;235.680;15.800;0.000;1.000;17.000\n',
 '16/12/2006;17:29:00;3.520;0.522;235.020;15.000;0.000;2.000;17.000\n',
 '16/12/2006;17:30:00;3.702;0.520;235.090;15.800;0.000;1.000;17.000\n',
 '16/12/2006;17:31:00;3.700;0.520;235.220;15.800;0.000;1.000;17.000\n',
 '16/12/2006;17:32:00;3.668;0.510;233.990;15.800;0.000;1.000;17.000\n']